In [1]:
# %matplotlib inline
import os
import sys
# import matplotlib
import time

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
#import networkx as nx
import pickle
from collections import defaultdict

#from datetime import datetime

from sktensor import dtensor, cp_als
# matplotlib.style.use('ggplot')

### Creating a list of all edges

In [3]:
#with open('dblp_inpr.xml', 'rb') as rf:
with open('dblp.xml', 'rb') as rf:
    readfile = rf.read().split('<inproceedings')

In [4]:
nb_edges = len(readfile)
print('%d edges in the file') %(nb_edges)

# Only take 1000 edges (publications)
# nb_edges = 10001

1761352 edges in the file


### Creating an empty dataframe to host the edges attributes

In [6]:
columns = ['author', 'year', 'crossref']
df = pd.DataFrame(index   = 5*range(nb_edges),
                  columns = columns)

In [7]:
df.head()

,author,year,crossref
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


### Filling the dataframe with the actual edges attributes

In [11]:
temp = time.time()
counter = 0
nb_authors = 0

for i in range(1,nb_edges):
    if i % 100000 == 1:
        print('%d/%d edges treated - %.0f seconds elapsed') %(i, nb_edges, time.time() - temp)
   
    contents = readfile[i].splitlines()
    author_list = []
    year = ''
    cross = ''
    
    if contents != '':
        for content in contents:
            if content[:8] == '<author>':
                author_list += [content[8:-9]]
            elif content[:6] == '<year>':
                year = content[6:10]
            elif content[:10] == '<crossref>':
                conf = content.rsplit('/', )
                cross = conf[1]                
            if (i == nb_edges -1)&(content[:16] == '</inproceedings>') :
                break
        
        # Consider only the publications of type inproceedings between 1991 and 2007 :
        #if ((int(year[:4]) >= 1991)and(int(year[:4]) <= 2007)) : 
        for k in range(len(author_list)):
            df.iat[counter, 0] = author_list[k]
            df.iat[counter, 1] = int(year)
            df.iat[counter, 2] = cross
            counter += 1
        nb_authors += len(author_list)

    else:
        df = df.ix[df.index != i]        

1/1761352 edges treated - 0 seconds elapsed
100001/1761352 edges treated - 138 seconds elapsed
200001/1761352 edges treated - 149 seconds elapsed
300001/1761352 edges treated - 161 seconds elapsed
400001/1761352 edges treated - 172 seconds elapsed
500001/1761352 edges treated - 185 seconds elapsed
600001/1761352 edges treated - 198 seconds elapsed
700001/1761352 edges treated - 209 seconds elapsed
800001/1761352 edges treated - 222 seconds elapsed
900001/1761352 edges treated - 233 seconds elapsed
1000001/1761352 edges treated - 244 seconds elapsed
1100001/1761352 edges treated - 257 seconds elapsed
1200001/1761352 edges treated - 270 seconds elapsed
1300001/1761352 edges treated - 282 seconds elapsed
1400001/1761352 edges treated - 295 seconds elapsed
1500001/1761352 edges treated - 307 seconds elapsed
1600001/1761352 edges treated - 319 seconds elapsed
1700001/1761352 edges treated - 332 seconds elapsed


IndexError: index 8806760 is out of bounds for axis 0 with size 8806760

In [9]:
df.year.min(), df.year.max()

(1991, 2007)

In [9]:
df.reset_index(inplace=True)

In [10]:
df = df[df.index < nb_authors]

In [11]:
df.drop(['index'], axis=1, inplace=True)

### Keep authors with at least 10 publications

In [14]:
author_list = pd.unique(df.iloc[:,0])

In [15]:
# Get the indexes of each author
temp = time.time()
count = 0

authors_indexes = [(element, index) for index, element in enumerate(df.author)]
author_index = defaultdict(list)

for k, v in authors_indexes:
    if count % 100000 == 1:
        print('%d/%d lines treated - %.0f seconds elapsed') %(count, len(author_list), time.time() - temp)
  
    author_index[k].append(v)
    
    count += 1
    
# Keep the 10+ publications
df = df[df.index.isin(author_index_sorted)]

1/738436 lines treated - 1 seconds elapsed
100001/738436 lines treated - 4 seconds elapsed
200001/738436 lines treated - 4 seconds elapsed
300001/738436 lines treated - 4 seconds elapsed
400001/738436 lines treated - 4 seconds elapsed
500001/738436 lines treated - 4 seconds elapsed
600001/738436 lines treated - 5 seconds elapsed
700001/738436 lines treated - 5 seconds elapsed
800001/738436 lines treated - 5 seconds elapsed
900001/738436 lines treated - 6 seconds elapsed
1000001/738436 lines treated - 6 seconds elapsed
1100001/738436 lines treated - 6 seconds elapsed
1200001/738436 lines treated - 6 seconds elapsed
1300001/738436 lines treated - 6 seconds elapsed
1400001/738436 lines treated - 7 seconds elapsed
1500001/738436 lines treated - 7 seconds elapsed
1600001/738436 lines treated - 7 seconds elapsed
1700001/738436 lines treated - 7 seconds elapsed
1800001/738436 lines treated - 7 seconds elapsed
1900001/738436 lines treated - 7 seconds elapsed
2000001/738436 lines treated - 7 se

### Dumping the result in a file

In [34]:
pickle.dump(df, open("dblp_inproceeding_10.p", "wb"))

In [2]:
df = pickle.load(open("dblp_inproceeding_10.p", "rb"))

In [3]:
len(pd.unique(df.author)), len(pd.unique(df.crossref))

(56213, 2521)

In [4]:
df.head()

,author,year,crossref
2,Roger King,1993,lncs
4,Yelena Yesha,1993,lncs
5,Jean-Louis Imbert,1994,lncs
6,Jochen D&ouml;rre,1991,lncs
8,Stanley Y. W. Su,1993,lncs


In [9]:
df = df[df.crossref != '']
len(pd.unique(df.ix[df.year == 2007, 0]))

40851

In [10]:
len(set(df.ix[df.year == 2007, 0]).intersection(df.ix[(df.year <= 2006)&(df.year >= 1997), 0]))

40848

In [14]:
val = 6
df_train = df[df.year>=1991+val]
df_train = df[df.year<=2000+val]
df_test = df[df.year==2001+val]

df_train.reset_index(inplace=True)
df2 = df_train.iloc[:,1:]
df_train = df2
del df2

In [15]:
author_list = pd.unique(df_train.iloc[:,0])

# Get the indexes of each author
authors_indexes = [(element, index) for index, element in enumerate(df_train.author)]
author_index = defaultdict(list)

for k, v in authors_indexes:
    author_index[k].append(v)
    
# Keep the authors with more than 10 publications
author_index_sorted = []
count = 0

for author in author_list : 
    if len(author_index[author]) > 10 :
        author_index_sorted += author_index[author]
        count += len(author_index[author])

In [16]:
df_train = df_train[df_train.index.isin(author_index_sorted)]
df_test = df_test[df_test.author.isin(df_train.author)]
df_test = df_test[df_test.crossref.isin(df_train.crossref)]
df_train.shape, df_test.shape

((1024192, 3), (107055, 3))

In [17]:
len(pd.unique(df_test.ix[df_test.year == 2007, 0]))

29390

In [19]:
len(pd.unique(df_test.ix[df_test.year == 2007, 2]))

1109